In [4]:
import math

def probabiliy_list(values):
    count_values = len(values)
    value_list = []
    prob_list = []
    different_val = 0
    for i in range(count_values):
        if (values[i] not in value_list): 
            value_list.append(values[i])
            prob_list.append(1)
            different_val +=1
        else:
            for j in range(different_val):
                if(values[i]==value_list[j]): prob_list[j]+=1
    for k in range(different_val):
        prob_list[k] = float(prob_list[k]) / count_values
    return prob_list 

def entropy(values):
    prob_list = probabiliy_list(values)
    different_values = len(prob_list)
    sum = 0
    for i in range(different_values):
        sum -= math.log(prob_list[i], 2) * prob_list[i]
    return sum

def gini(values):
    prob_list = probabiliy_list(values)
    different_values = len(prob_list)
    sum = 0
    for i in range(different_values):
        sum += prob_list[i]*(1-prob_list[i])
    return sum



In [5]:
list = [1,1,2,2]
print(entropy(list))
print(gini(list))
list2 = [1,1,1,2]
print(entropy(list2))
print(gini(list2))


1.0
0.5
0.8112781244591328
0.375


In [3]:
#this function needs to be implemented in the tree class
#class has variables: data, children
#if leaf: data is label otherwise data is featureindex
#children is a list with elements of form (value, node)
#Nodes save sorted listed of values --> check < on everyone except last
def predict(x):
    count_values = len(children)
    for i in range(count_values):
        if (not children):                          #empty children list --> leave
            return data
        elif(i == count_values-1):                  #Last value in list --> x > elements in this tree
            (children[i])[1].predict(x)
        elif(x[data] < (children[i])[0]):    #search child with right value
            (children[i])[1].predict(x)             #use predict on child recursivly